**크롤링 된 csv파일을 전처리 후 인코딩된 matrix로 변환한다**

# 0. import, 함수정의

In [1]:
import pandas as pd
import numpy as np
import re
import joblib
from konlpy.tag import *
from modules import *
from sklearn.feature_extraction.text import TfidfVectorizer

In [2]:
pip install joblib

Note: you may need to restart the kernel to use updated packages.


# 1. csv 및 필요한 파일 불러오기

In [3]:
# csv
df_a=pd.read_csv("appended/new.csv")

# 사용자 사전
komoran=Komoran(userdic='userdict.txt')

# 품사 리스트
pos=pd.read_csv('pos_table.txt', header=None, names='p')
poslist=pos.p.tolist()

# 불용어 리스트
stopwords=pd.read_excel("stopwords.xlsx")
stoplist=np.array(stopwords.words.tolist())

# 2. 의미없는 부분 제거

In [4]:
#전체 행에 전처리 적용
print('cleaning...')
df_a['content']=df_a.content.apply(cleaning)

#확인
#print(df_a.content.values[0])

cleaning...


# 3. 형태소 분석기 이용해 토큰화

In [5]:
print('tokenizing...')

tokenizing...


In [6]:
# 전체 행에 품사 부착 -> 각 행은 ('word', '품사') 튜플의 리스트 <-오래걸림 
df_a['content']=df_a.content.apply(komoran.pos)

In [7]:
# 전체 행에 품사추출함수 적용
df_a['content']=df_a.content.apply(PosPicker)

In [8]:
# 전체 행을 다시 형태소 리스트로 변환
df_a['content']=df_a.content.apply(komoran.morphs)

# 4. 불용어 제거

In [9]:
# 전체 행에 불용어 제거 함수 적용
print('erasing stopwords...')
df_a['content']=df_a.content.apply(DelStops)

#확인
#print(df_a.content[0])

erasing stopwords...


In [10]:
df_a.isad

0      1.0
1      1.0
2      1.0
3      1.0
4      1.0
      ... 
504    0.0
505    0.0
506    0.0
507    0.0
508    0.0
Name: isad, Length: 509, dtype: float64

# 5. 정수 인코딩

In [11]:
# 전체 행에 대해 문장화 적용
df_a.content=df_a.content.apply(wordjoin)

In [12]:
print('encoding...')

# TFID 생성
tfid=TfidfVectorizer()

# 문장 벡터화 (TFID)
tfid.fit_transform(df_a.content)
print("number of encoded features :", len(tfid.get_feature_names()))

# matrix로 저장 (TFID) -> 문장 수 * 단어 수 크기의 matrix가 생성된다
feature_matrix2=tfid.transform(df_a.content)
print("dimension of feature_matrix :",feature_matrix2.shape)

encoding...
number of encoded features : 9762
dimension of feature_matrix : (509, 9762)


NameError: name 'joblib' is not defined

In [15]:
# tfid 저장
joblib.dump(tfid, 'tfid/tfid_test.pkl')

['tfid/tfid_test.pkl']

In [ ]:
# matrix에 img, sticker, video, tags 열을 추가
print("adding columns...")
new_matrix=np.zeros((feature_matrix2.shape[0], feature_matrix2.shape[1]+4))

new_matrix[:,:-4]=feature_matrix2.toarray()
new_matrix[:,-4]=df_a.img
new_matrix[:,-3]=df_a.sticker
new_matrix[:,-2]=df_a.video
new_matrix[:,-1]=df_a.tags

print("4 columns added successfully :", new_matrix.shape)

# 6. matrix 저장

In [ ]:
np.save("matrix/new_matrix", new_matrix)